In [12]:
import os 
import re
import time
import pandas as pd
import multiprocessing as mp
import numpy as np
import dask.dataframe as dd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import torch.nn.functional as F
from torch import nn
import math
import matplotlib.pyplot as plt

from statistics import mean

import torch
from torch import optim

path0 ='../'
df = pd.read_csv(path0+"Dataset_S022Final.csv")

#df=df[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','sentPK','duration(ms)','droppedPKWrongPort','size','throughput','label']]   #,'channel'      
#df=df[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','droppedPKWrongPort','sentPK','size','DataQueueLen','passedUpPk','sentDownPK','DropPKByQueue','snir','throughput','label']]  

df=df[['Next_Current_diff','Next_Pre_diff','SNext_Current_diff','SNext_Pre_diff','rcvdPK','duration(ms)','droppedPKWrongPort','sentPK','size','channel','DataQueueLen','passedUpPk','rcvdPKFromLL','sentDownPK','DropPKByQueue','snir','throughput','label']]         


df_Normal=df[0:76064].copy()
#print(df_Normal)
df_UDP=df[76064:214037].copy()
#print(df_UDP)
df_pluies=df[214037:426866].copy()

#print(df_Normal2)
df_jam=df[462481:-1].copy()

X=df_Normal.drop(columns = ['label']).copy()
y=df_Normal[['label']].copy()
#print(y.columns)
X1=df_UDP.drop(columns = ['label']).copy()
y1=df_UDP[['label']].copy()
X2=df_pluies.drop(columns = ['label']).copy()
y2=df_pluies[['label']].copy()
X3=df_jam.drop(columns = ['label']).copy()
y3=df_jam[['label']].copy()

train=0.4


xcol=X.columns
ycol=y.columns


X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=train,shuffle=False)
X_train1, X_rem1, y_train1, y_rem1 = train_test_split(X1,y1, train_size=train,shuffle=False)
X_train2, X_rem2, y_train2, y_rem2 = train_test_split(X2,y2, train_size=train,shuffle=False)
X_train3, X_rem3, y_train3, y_rem3 = train_test_split(X3,y3, train_size=train,shuffle=False)

validation=0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5,shuffle=False)
X_valid1, X_test1, y_valid1, y_test1 = train_test_split(X_rem1,y_rem1, test_size=0.5,shuffle=False)
X_valid2, X_test2, y_valid2, y_test2 = train_test_split(X_rem2,y_rem2, test_size=0.5,shuffle=False)
X_valid3, X_test3, y_valid3, y_test3 = train_test_split(X_rem3,y_rem3, test_size=0.5,shuffle=False)


X_train=np.concatenate((X_train, X_train1, X_train2,X_train3))
X_valid=np.concatenate((X_valid, X_valid1, X_valid2,X_valid3))
X_test=np.concatenate((X_test, X_test1, X_test2,X_test3))

y_train=np.concatenate((y_train, y_train1, y_train2,y_train3))
y_valid=np.concatenate((y_valid, y_valid1, y_valid2,y_valid3))
y_test=np.concatenate((y_test, y_test1, y_test2,y_test3))

X_train = pd.DataFrame(X_train, columns= xcol)
X_valid = pd.DataFrame(X_valid, columns= xcol)
X_test = pd.DataFrame(X_test, columns= xcol)

y_train = pd.DataFrame(y_train, columns= ycol)
y_valid = pd.DataFrame(y_valid, columns= ycol)
y_test = pd.DataFrame(y_test, columns= ycol)




In [13]:
X_train1=X_train.values
y_train1=y_train.values
print(y_train1)

torch_tensor = torch.tensor(X_train1)
t=[]

for j in y_train1:
    i=j[0]
#    print(i)
    if (i=='Normal'):
        t.append(int(0))
    if (i=='DDOS_UDP_FLOOD'):
        t.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        t.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        t.append(int(3))

labels = torch.LongTensor(t)
print(labels)

bigX = X_valid
bigY = y_valid['label']
#print(bigY)
X_valid1=bigX.values
y_valid1=bigY.values


vtorch_tensor = torch.tensor(X_valid1)
v=[]

for i in y_valid1:
    #i=j[0]
    #print(i)
    if (i=='Normal'):
        v.append(int(0))
        #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        v.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        v.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        v.append(int(3))

vlabels = torch.LongTensor(v)


X_test1=X_test.values
y_test1=y_test.values


ttorch_tensor = torch.tensor(X_test1)
#torch_tensor[torch_tensor != torch_tensor]=float(0)
t=[]

for j in y_test1:
    i=j[0]
#    print(i)
    if (i=='Normal'):
        t.append(int(0))
        #t.append(int(1))
    if (i=='DDOS_UDP_FLOOD'):
        t.append(int(1))
    if (i=='PLUIES_ET_ORAGES'):
        t.append(int(2))
    if (i=='BROUILLAGE_Trafic'):
        t.append(int(3))

tlabels = torch.LongTensor(t)



[['Normal']
 ['Normal']
 ['Normal']
 ...
 ['Normal']
 ['BROUILLAGE_Trafic']
 ['Normal']]
tensor([0, 0, 0,  ..., 0, 3, 0])


In [18]:
torch.set_default_dtype(torch.float64)

from torch import nn, optim
import torch.nn.functional as F

class My_module(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,8,3,padding=1)
        self.conv2=nn.Conv2d(8,12,3,padding=1)
        self.conv3=nn.Conv2d(12,18,3,padding=1)
        self.fc1 = nn.Linear(18, 30)
        self.fc2 = nn.Linear(30,56)
        self.fc3 = nn.Linear(56, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(p=0.0)
        self.pool=nn.MaxPool2d(1,1)
        
    def forward(self, x):
        # make sure input tensor is flattened
        #x = x.view(x.shape[0], -1)
        #print(x.shape)
        
        #x=self.pool(F.relu(self.conv1(x)))
        #print(x.shape)

        #x=self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        
        #x=self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        
        #x=x.view(-1,18*4*4)
        #print(x.shape)
        x = self.dropout(F.relu(self.fc1(x)))
        #print(x.shape)
        
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x))) 
        x = self.dropout(F.relu(self.fc4(x))) 
    
        x = F.log_softmax(self.fc5(x), dim=1)
        #print(x.shape)
        
        return x
class My_module(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,8,3,padding=1)
        self.conv2=nn.Conv2d(8,12,3,padding=1)
        self.conv3=nn.Conv2d(12,18,3,padding=1)
        self.fc1 = nn.Linear(18*6*3, 350)
        self.fc2 = nn.Linear(350,400)
        self.fc3 = nn.Linear(56, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(400, 2)
        self.dropout = nn.Dropout(p=0.0)
        self.pool=nn.MaxPool2d(1,1)
        
    def forward(self, x):
        # make sure input tensor is flattened
        #x = x.view(x.shape[0], -1)
        #print(x.shape)
        
        x=self.pool(F.relu(self.conv1(x)))
        #print(x.shape)

        x=self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        
        x=self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        
        x=x.view(-1,18*6*3)
        #print(x.shape)
        x = self.dropout(F.relu(self.fc1(x)))
        #print(x.shape)
        
        x = self.dropout(F.relu(self.fc2(x)))
        #x = self.dropout(F.relu(self.fc3(x))) 
        #x = self.dropout(F.relu(self.fc4(x))) 
    
        x = F.log_softmax(self.fc5(x), dim=1)
        #print(x.shape)
        
        return x    
model = My_module()
    
Fonction_de_perte = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

model.load_state_dict(torch.load('models\\dataset1_1_50_CNN.pth'))
#indices = [0,3, 299:303]


RuntimeError: Error(s) in loading state_dict for My_module:
	size mismatch for fc1.weight: copying a param with shape torch.Size([350, 18]) from checkpoint, the shape in current model is torch.Size([350, 324]).
	size mismatch for fc5.weight: copying a param with shape torch.Size([4, 400]) from checkpoint, the shape in current model is torch.Size([2, 400]).
	size mismatch for fc5.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).

In [142]:

vtorch_tensor=torch.reshape(vtorch_tensor,(70010,1,4,4))
ttorch_tensor=torch.reshape(ttorch_tensor,(125010,1,4,4))
torch_tensor=torch.reshape(torch_tensor,(100015,1,4,4))


torch.Size([125010, 1, 4, 4])
torch.Size([100015, 1, 4, 4])
tensor([[[2.1728e-03, 2.5947e-03, 2.3385e-03, 4.5851e-03],
         [0.0000e+00, 2.3810e-01, 0.0000e+00, 2.4473e-03],
         [1.0002e-01, 0.0000e+00, 9.1377e-03, 1.9384e-04],
         [1.8672e-02, 0.0000e+00, 2.4010e-02, 8.6110e-04]]])


In [36]:
#labels1=labels[loc]
a=time.time()
inputs=torch_tensor[:,:] 
labels1=labels[:]
print(len(inputs))
print(len(labels1))
vinputs =vtorch_tensor[:,:] 
vlabels1=vlabels[:]

epochs = 10
valid_loss=0
accuracy=0
valid_loss_min=np.Inf

####33  !!!!!!!!!! This shit cost alot of time ----> do not use it 
#b=time.time()
#with torch.no_grad():
   # for i, x in enumerate(vinputs):
       # x2=x[None,:]
       # output = torch.exp(model(x2))
       # l2=vlabels1[i][None]
       # test_loss+=Fonction_de_perte(output, l2)
     #   output = torch.exp(output)
      #  top_p , top_c = output.topk(1, dim=1)
     #   equals = top_c==vlabels1[i].view(*top_c.shape)
    #    accuracy += torch.mean(equals.type(torch.FloatTensor))

   # tain_losses.append(running_loss/len(inputs))
   # test_losses.append(test_loss/len(vinputs))
   # print ('test accuracy :{0}, test loss : {2} ,  time {1} '.format(accuracy*100/len(vinputs) , time.time()-b ,test_loss/len(vinputs) ))

######## ---> use this :) 
b=time.time()
with torch.no_grad():
    output = torch.exp(model(vinputs))
    valid_loss=Fonction_de_perte(output, vlabels1)
top_p , top_c = output.topk(1, dim=1)
propabilities= torch.mean(top_p.type(torch.FloatTensor))*100
equals = top_c==vlabels1.view(*top_c.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
#print ('valid accuracy :{0:.8f} with total prob : {3:.8f} and  valid loss : {2:.8f} ,  time {1:.6f} '.format(accuracy*100 , time.time()-b ,test_loss,propabilities))
print ('Epoch  : valid accuracy :{0:.8f} with total prob : {1:.8f} '.format(accuracy*100  ,propabilities))

####
#print(inputs.shape[0])
train_losses=[]
valid_losses=[]
for e in range(epochs):
    running_loss = 0
    for i, x in enumerate(inputs):
        optimizer.zero_grad()
        x2=x[None,:]
        output = model.forward(x2)
        l2=labels1[i][None]
        #print(l2)
        loss = Fonction_de_perte(output, l2)
      
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    else:
        accuracy=0
        #print(l2)
  
        b=time.time()
        with torch.no_grad():
            ##
            #with torch.no_grad():
            #for i, x in enumerate(vinputs):
             #   x2=x[None,:]
              #  output = torch.exp(model(x2))
               # l2=vlabels1[i][None]
                #valid_loss1+=Fonction_de_perte(output, l2)
            #test_loss=test_loss/len(vinputs)
            ##
                
            model.eval()
            output = torch.exp(model(vinputs))
            valid_loss=Fonction_de_perte(output, vlabels1)
        top_p , top_c = output.topk(1, dim=1)
        propabilities= torch.mean(top_p.type(torch.FloatTensor))*100
        equals = top_c==vlabels1.view(*top_c.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        model.train()
        print('Epoch {2} : Training loss {0:.8f} and valid loss : {1:.8f} :  '.format(running_loss/len(inputs),valid_loss, e))
   
        print ('Epoch {2} : valid accuracy :{0:.8f} with total prob : {1:.8f} '.format(accuracy*100  ,propabilities, e))
        #print('validation loss with iterations {0}'.format(valid_loss1/len(vinputs) ) )   
        train_losses.append(running_loss/len(inputs))
        valid_losses.append(valid_loss)
        if (valid_loss<valid_loss_min):
            print('validation loss decreased , saving model ({:.8f} ==> {:.8f})'.format(valid_loss_min,valid_loss))
            torch.save(model.state_dict(),'models\\mod_temp4.pth')
            valid_loss_min=valid_loss
print(time.time()-a)

346939
346939
Epoch  : valid accuracy :13.32329082 with total prob : 26.40723228 


KeyboardInterrupt: 

In [ ]:
print(output)
print(output.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

#test_losses_scaled = scaler.fit_transform(test_losses)
plt.ylim([-1,1])

plt.plot(train_losses, label='train loss ')
plt.plot(valid_losses, label='valid loss ')
plt.legend(frameon=False)

In [ ]:
Tr=np.array(train_losses)
Te=np.array(valid_losses)

Tr=np.reshape(Tr, (len(Tr),1))
Te=np.reshape(Te, (len(Te),1))

# fit on training data column
scale = StandardScaler().fit(Tr)
tain_losses = scale.transform(Tr)
scale = StandardScaler().fit(Te)
test_losses = scale.transform(Te)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

#test_losses_scaled = scaler.fit_transform(test_losses)
plt.ylim([-1,1])

plt.plot(tain_losses, label='Train loss')
plt.plot(test_losses, label='valid loss')
plt.legend(frameon=False)

In [350]:
model.load_state_dict(torch.load('models\\mod_temp3.pth'))

<All keys matched successfully>

In [140]:
a=time.time()
summm=[]
sum1=[]
sum2=[]
sum3=[]
sum4=[]

tinputs =ttorch_tensor[:,:] 
tlabels1=tlabels[:]


#tinputs =torch_tensor[:,:] 
#tlabels1=labels[:]

print('=====================')
b=time.time()
output = torch.exp(model(tinputs))
test_loss=Fonction_de_perte(output, tlabels1)
top_p , top_c = output.topk(1, dim=1)
propabilities= torch.mean(top_p.type(torch.FloatTensor))*100
equals = top_c==tlabels1.view(*top_c.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
model.train()
print ('test accuracy :{0:.8f} with total prob : {3:.8f} and  test loss : {2:.8f} ,  time {1:.8f} '.format(accuracy*100 , time.time()-b ,test_loss ,propabilities))

class_correct = list(0. for i in range (4))
class_total = list(0. for i in range (4))

C_n_udp=0
C_n_pluies=0
C_n_jam=0
C_n_total=0

C_j_normal=0
C_j_udp=0
C_j_pluies=0
C_j_total=0

for i, x in enumerate(tinputs):
    optimizer.zero_grad()
    x2=x[None,:]
    with torch.no_grad():
        output = torch.exp(model(x2))
    out=output.detach().numpy()*100
 
    l3=tlabels1[i].item()
  
    if(l3==0):
        summm.append(out[0][0])
        sum1.append(out[0][0])
        if(top_c[i][0]!=l3):
            if(top_c[i][0]==1):
                C_n_udp+=1
            if(top_c[i][0]==2):
                C_n_pluies+=1
            if(top_c[i][0]==3):
                C_n_jam +=1
            
            C_n_total +=1
            
    if(l3==1):
        summm.append(out[0][1])
        sum2.append(out[0][1])
    if(l3==2):
        summm.append(out[0][2])
        sum3.append(out[0][2])
    if(l3==3):
        summm.append(out[0][3])
        sum4.append(out[0][3])
        if(top_c[i][0]!=l3):
            if(top_c[i][0]==1):
                C_j_udp+=1
            if(top_c[i][0]==2):
                C_j_pluies+=1
            if(top_c[i][0]==0):
                C_j_normal +=1
            
            C_j_total +=1

    class_total[l3]+=1
    class_correct[l3]+=equals[i][0]
    #print(output)
print('la précision de detection globale: {0} '.format(mean(summm)))
print('detection des communication normal: {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum1), class_correct[0]*100/class_total[0] ,class_correct[0] ,class_total[0]))

#print('details normal classed udp :{0:.5f}, pluies {1:.5f} , jam {2:.5f} (ou {3:.5f},{4:.5f},{5:.5f}) '.format(C_n_udp*100/class_total[0],C_n_pluies*100/class_total[0],C_n_jam*100/class_total[0],C_n_udp*100/C_n_total,C_n_pluies*100/C_n_total,C_n_jam*100/C_n_total))
print('detection du deni de service par udp flood {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum2), class_correct[1]*100/class_total[1] ,class_correct[1] ,class_total[1]))
print('detection du deni naturel : pluies et orages : {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum3), class_correct[2]*100/class_total[2] ,class_correct[2] ,class_total[2]))
print('detection du deni naturel : jam : {0:.8f} with accuracy {1:.8f} ( {2:.0f}/{3:.0f} )'.format(mean(sum4), class_correct[3]*100/class_total[3] ,class_correct[3] ,class_total[3]))
print('details jam classed udp :{0:.5f}, pluies {1:.5f} , normal {2:.5f} (ou {3:.5f},{4:.5f},{5:.5f}) '.format(C_j_udp*100/class_total[3],C_j_pluies*100/class_total[3],C_j_normal*100/class_total[3],C_j_udp*100/C_j_total,C_j_pluies*100/C_j_total,C_j_normal*100/C_j_total))


print('=====================')

print('<================================>')
print('Total_time')
print(time.time()-a)


test accuracy :81.93904877 with total prob : 81.87981415 and  test loss : -0.72699457 ,  time 174.84038281 
la précision de detection globale: 72.69945739686045 
detection des communication normal: 80.77099487 with accuracy 99.69945234 ( 89566/89836 )
detection du deni de service par udp flood 99.98543633 with accuracy 100.00000000 ( 12866/12866 )
detection du deni naturel : pluies et orages : 17.11094296 with accuracy 0.00000000 ( 0/5393 )
detection du deni naturel : jam : 26.80012619 with accuracy 0.00000000 ( 0/16915 )
details jam classed udp :0.00000, pluies 0.00000 , normal 100.00000 (ou 0.00000,0.00000,100.00000) 
<================================>
Total_time
337.22450590133667


In [63]:
vinputs =vtorch_tensor[:, :] 
vlabels1=vlabels[:]

output = torch.exp(model(vinputs))
top_p , top_c = output.topk(1, dim=1)
print(top_p)
print(top_c)
equals = top_c==vlabels1.view(*top_c.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
propabilities= torch.mean(top_p.type(torch.FloatTensor))
print(propabilities)
print (accuracy)

tensor([[1.0000],
        [0.6282],
        [0.7119],
        ...,
        [0.9976],
        [1.0000],
        [1.0000]], grad_fn=<TopkBackward>)
tensor([[0],
        [3],
        [3],
        ...,
        [2],
        [0],
        [0]])
tensor(0.9808, dtype=torch.float32, grad_fn=<MeanBackward0>)
tensor(0.9793, dtype=torch.float32)


In [69]:
#tester la qualite du dataset en terme de quantite
df00 = pd.read_csv("src/cvs/datasets1/small/Dataset_S02Final.csv")
#df00 = pd.read_csv("src/cvs/datasets1/small/testbed/Train_test/y.csv")

#print(df)
df111=df00.to_numpy()
total=len(df111)
n=0
u=0
p=0
j=0
for i in enumerate(df111):
    if(i[1][-1]=='Normal'):
        n+=1

    if(i[1][-1]=='PLUIES_ET_ORAGES'):
        p+=1
    if(i[1][-1]=='DDOS_UDP_FLOOD'):
        u+=1
    if(i[1][-1]=='BROUILLAGE_Trafic'):
        j+=1
print('pourcentge du trafic normal :'+str(n/total *100))
print('pourcentge du trafic udp_flood :'+str(u/total *100))
print('pourcentge du trafic dans un mauvais temps :'+str(p/total *100))
print('pourcentge du trafic vicitme d\' brouillage :'+str(j/total *100))

#print(n+u+p+j)
#print(total)

pourcentge du trafic normal :59.453586205955666
pourcentge du trafic udp_flood :23.12923048737943
pourcentge du trafic dans un mauvais temps :14.837523947993631
pourcentge du trafic vicitme d' brouillage :2.579659358671269


In [70]:
pourcentge du trafic normal :21.765924563089197
pourcentge du trafic udp_flood :26.890551578569884
pourcentge du trafic dans un mauvais temps :41.47977649188935
pourcentge du trafic vicitme d' brouillage :9.863747366451566

pourcentge du trafic normal :52.16949819817537
pourcentge du trafic udp_flood :23.12923048737943
pourcentge du trafic dans un mauvais temps :14.837523947993631
pourcentge du trafic vicitme d' brouillage :9.863747366451566

SyntaxError: invalid syntax (<ipython-input-70-b734b4a253c9>, line 1)

In [35]:
#df2=df1[462481:-1].copy()
minn=df2[0][1]
maxx=df2[-1][1]
print(minn)
print(maxx)
#print(len(df2))
df3=df2.copy()


280.000421818582
487.2262257424459


In [36]:
for i in enumerate(df3):
    i[1][1]=i[1][1]-minn+maxx
    i[1][2]=i[1][2]-minn+maxx
    i[1][3]=i[1][3]-minn+maxx
    
df4=np.concatenate((df2,df3))
minn=df4[0][1]
maxx=df4[-1][1]
print(minn)
print(maxx)

280.000421818582
694.4520296663097


In [37]:
df2=df4.copy()

In [39]:

df=df1[0:462481].copy()
df=np.concatenate((df,df2))
minn=df[0][1]
maxx=df[-1][1]
print(minn)
print(maxx)


0.003650631698
694.4520296663097


In [42]:
df = pd.DataFrame(df, columns= col)

df.to_csv ('src/cvs/datasets1/Dataset_03Final.csv', index=False , header=True)


In [9]:
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())



torch.save({
            'epochs': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'loss': loss
            },  'models\\mod1.pth')


#checkpoint = {'input_size': 11,
#              'output_size': 4,
 #             'hidden_layers': [each.out_features for each in model.hidden_layers],
#              'state_dict': model.state_dict()}

#torch.save(checkpoint, 'models\\mod1.pth')



Our model: 

 My_module(
  (fc1): Linear(in_features=11, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=56, bias=True)
  (fc5): Linear(in_features=56, out_features=4, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
) 

The state dict keys: 

 odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias', 'fc5.weight', 'fc5.bias'])


NameError: name 'epochs' is not defined

In [10]:
model = My_module()
print("Our model: \n\n", model, '\n')

optimizer = optim.SGD(model.parameters(), lr=0.002)

Loaded = torch.load('models\\mod1.pth')
model.load_state_dict(Loaded['model_state_dict'])
optimizer.load_state_dict(Loaded['optimizer_state_dict'])
epochs = Loaded['epochs']
#loss = checkpoint['loss']

model.eval()
# - or -
model.train()

Our model: 

 My_module(
  (fc1): Linear(in_features=11, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=56, bias=True)
  (fc5): Linear(in_features=56, out_features=4, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
) 



My_module(
  (fc1): Linear(in_features=11, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=56, bias=True)
  (fc5): Linear(in_features=56, out_features=4, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [92]:
#labels1=labels[loc]
a=time.time()
inputs=torch_tensor[:,:] 
labels1=labels[:]

vinputs =vtorch_tensor[:,:] 
vlabels1=vlabels[:]

epochs = 5
test_loss=0
accuracy=0

####33  !!!!!!!!!! This shit cost alot of time ----> do not use it 
#b=time.time()
#with torch.no_grad():
   # for i, x in enumerate(vinputs):
       # x2=x[None,:]
       # output = torch.exp(model(x2))
       # l2=vlabels1[i][None]
       # test_loss+=Fonction_de_perte(output, l2)
     #   output = torch.exp(output)
      #  top_p , top_c = output.topk(1, dim=1)
     #   equals = top_c==vlabels1[i].view(*top_c.shape)
    #    accuracy += torch.mean(equals.type(torch.FloatTensor))

   # tain_losses.append(running_loss/len(inputs))
   # test_losses.append(test_loss/len(vinputs))
   # print ('test accuracy :{0}, test loss : {2} ,  time {1} '.format(accuracy*100/len(vinputs) , time.time()-b ,test_loss/len(vinputs) ))

######## ---> use this :) 
b=time.time()
with torch.no_grad():
    output = torch.exp(model(vinputs))
    test_loss=Fonction_de_perte(output, vlabels1)
top_p , top_c = output.topk(1, dim=1)
propabilities= torch.mean(top_p.type(torch.FloatTensor))*100
equals = top_c==vlabels1.view(*top_c.shape)
accuracy = torch.mean(equals.type(torch.FloatTensor))
print ('test accuracy :{0:.8f} with total prob : {3:.8f} and  test loss : {2:.8f} ,  time {1:.6f} '.format(accuracy*100 , time.time()-b ,test_loss,propabilities))


####
#print(inputs.shape[0])
tain_losses=[]
test_losses=[]
for e in range(epochs):
    running_loss = 0
    for i, x in enumerate(inputs):
        optimizer.zero_grad()
        x2=x[None,:]
        output = model.forward(x2)
        l2=labels1[i][None]
        #print(l2)
        loss = Fonction_de_perte(output, l2)
      
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    else:
        test_loss=0
        accuracy=0
        #print(l2)
        print(f"Training loss: {running_loss/len(inputs)}")
   
        b=time.time()
        with torch.no_grad():
            ##
            #with torch.no_grad():
            for i, x in enumerate(vinputs):
                x2=x[None,:]
                output = torch.exp(model(x2))
                l2=vlabels1[i][None]
                #test_loss+=Fonction_de_perte(output, l2)
            #test_loss=test_loss/len(vinputs)
            ##
                
            model.eval()
            output = torch.exp(model(vinputs))
            test_loss=Fonction_de_perte(output, vlabels1)
        top_p , top_c = output.topk(1, dim=1)
        propabilities= torch.mean(top_p.type(torch.FloatTensor))*100
        equals = top_c==vlabels1.view(*top_c.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))
        model.train()
        print ('test accuracy :{0:.8f} with total prob : {3:.8f} and  test loss : {2:.8f} ,  time {1:.8f} '.format(accuracy*100 , time.time()-b ,test_loss ,propabilities))
            
        tain_losses.append(running_loss/len(inputs))
        test_losses.append(test_loss)

print(time.time()-a)

test accuracy :97.25601196 with total prob : 96.81060028 and  test loss : -0.95929247 ,  time 0.334789 


NameError: name 'clear' is not defined

In [86]:
c=torch.cuda.is_available()

In [77]:
print(c)

False
